<a href="http://laf-fabric.readthedocs.org/en/latest/" target="_blank"><img align="left" src="images/laf-fabric-xsmall.png"/></a>
<a href="http://www.godgeleerdheid.vu.nl/etcbc" target="_blank"><img align="left" src="images/VU-ETCBC-xsmall.png"/></a>
<a href="http://www.persistent-identifier.nl/?identifier=urn%3Anbn%3Anl%3Aui%3A13-048i-71" target="_blank"><img align="left"src="images/etcbc4easy-small.png"/></a>
<a href="http://tla.mpi.nl" target="_blank"><img align="right" src="images/TLA-xsmall.png"/></a>
<a href="http://www.dans.knaw.nl" target="_blank"><img align="right"src="images/DANS-xsmall.png"/></a>

# Phonological representation

Here comes the plain text of the Hebrew Bible in the phonological/phonetic representation by
Nicolai Winther-Nielsen et al.
This is a prerequisite for doing trigram analysis on the Hebrew text.

You can download these descriptions directly from my 
[SURFdrive](https://surfdrive.surf.nl/files/public.php?service=files&t=355dba3fbef111fc3ab8ac6554aaf85a).

In [33]:
import sys, collections, re
import unicodedata

from laf.fabric import LafFabric
from etcbc.preprocess import prepare
from etcbc.lib import Transcription
fabric = LafFabric()

  0.00s This is LAF-Fabric 4.5.3
API reference: http://laf-fabric.readthedocs.org/en/latest/texts/API-reference.html
Feature doc: http://shebanq-doc.readthedocs.org/en/latest/texts/welcome.html



In [2]:
version = '4b'
fabric.load('etcbc{}'.format(version), '--', 'pproduction', {
    "xmlids": {"node": False, "edge": False},
    "features": ('''
        otype
        g_word_utf8 g_cons_utf8 trailer_utf8
        g_word g_cons lex_utf8 lex
        sp vs vt gn nu ps st
        uvf prs
        language
        book chapter verse label
    ''',''),
    "primary": True,
    "prepare": prepare,
})
exec(fabric.localnames.format(var='fabric'))

  0.00s LOADING API: please wait ... 
  0.00s INFO: USING DATA COMPILED AT: 2015-06-29T05-30-49
  6.15s LOGFILE=/Users/dirk/SURFdrive/laf-fabric-output/etcbc4b/pproduction/__log__pproduction.txt
    16s ETCBC reference: http://laf-fabric.readthedocs.org/en/latest/texts/ETCBC-reference.html
  0.00s LOADING API with EXTRAs: please wait ... 
  0.00s INFO: USING DATA COMPILED AT: 2015-06-29T05-30-49
  0.68s INFO: DATA LOADED FROM SOURCE etcbc4b AND ANNOX -- FOR TASK pproduction AT 2015-08-27T12-28-30
  0.00s INFO: DATA LOADED FROM SOURCE etcbc4b AND ANNOX -- FOR TASK pproduction AT 2015-08-27T12-28-30


## Trailer

Before we generate the text, let's list all the different suffixes and their number of occurrences.

In [3]:
trailer = collections.defaultdict(int)

for node in F.otype.s('word'):
    trailer[F.trailer_utf8.v(node)] += 1

In [61]:
trans = Transcription()

# Bible OL words

In [92]:
unorm = 'NFD'
bolfile = infile('bolwords.txt')
bolwords = dict()
bolchars = dict()
for line in bolfile:
    (wid, wheb, wphono) = line.rstrip().split('=')
    nwheb = unicodedata.normalize(unorm, wheb)
    nwhparts = nwheb.split(' ')
    wphonoparts = nwheb.split(' ')
    for (i, nwhpart) in enumerate(nwhparts):
        for u in nwhpart:
            if u not in bolchars:
                bolchars[u] = unicodedata.name(u)
        bolwords[nwhpart] = wphonoparts[i]
for u in sorted(bolchars):
    print('in BOL: {:04X} [ {} ] {}'.format(ord(u), u, bolchars[u]))
delchars = set(u for u in Transcription.hebrew_mapping.values() if len(u) == 1) - set(bolchars.keys())
delchars_map = dict((ord(u), None) for u in delchars)
for u in sorted(delchars):
    print('NOT in BOL: {:04X} [ {} ] {}'.format(ord(u), u, unicodedata.name(u)))



in BOL: 05B0 [ ְ ] HEBREW POINT SHEVA
in BOL: 05B1 [ ֱ ] HEBREW POINT HATAF SEGOL
in BOL: 05B2 [ ֲ ] HEBREW POINT HATAF PATAH
in BOL: 05B3 [ ֳ ] HEBREW POINT HATAF QAMATS
in BOL: 05B4 [ ִ ] HEBREW POINT HIRIQ
in BOL: 05B5 [ ֵ ] HEBREW POINT TSERE
in BOL: 05B6 [ ֶ ] HEBREW POINT SEGOL
in BOL: 05B7 [ ַ ] HEBREW POINT PATAH
in BOL: 05B8 [ ָ ] HEBREW POINT QAMATS
in BOL: 05B9 [ ֹ ] HEBREW POINT HOLAM
in BOL: 05BB [ ֻ ] HEBREW POINT QUBUTS
in BOL: 05BC [ ּ ] HEBREW POINT DAGESH OR MAPIQ
in BOL: 05BD [ ֽ ] HEBREW POINT METEG
in BOL: 05BE [ ־ ] HEBREW PUNCTUATION MAQAF
in BOL: 05BF [ ֿ ] HEBREW POINT RAFE
in BOL: 05C1 [ ׁ ] HEBREW POINT SHIN DOT
in BOL: 05C2 [ ׂ ] HEBREW POINT SIN DOT
in BOL: 05D0 [ א ] HEBREW LETTER ALEF
in BOL: 05D1 [ ב ] HEBREW LETTER BET
in BOL: 05D2 [ ג ] HEBREW LETTER GIMEL
in BOL: 05D3 [ ד ] HEBREW LETTER DALET
in BOL: 05D4 [ ה ] HEBREW LETTER HE
in BOL: 05D5 [ ו ] HEBREW LETTER VAV
in BOL: 05D6 [ ז ] HEBREW LETTER ZAYIN
in BOL: 05D7 [ ח ] HEBREW LETTER HET
in BOL: 05D

In [94]:
tests_proto = '''
bl=שָּׁמַיִם=sh
et=שָּׁמַיִמ=sh
11229=הָיֽוּ=hāyû
11230=הָיוּ=hāyû
11231=הָיוּ=hāʸû
11146=הָֽיוּ=hāʸû
'''

test_data = dict()
tests = tests_proto.strip().split('\n')
for test in tests:
    (src, ustr, trans) = test.split('=')
    test_data[src] = ustr
    print('{} => {}'.format(src, trans))
    for u in ustr:
        print(unicodedata.name(u))

print(test_data['11230'] == test_data['11231'])

bl => sh
HEBREW LETTER SHIN
HEBREW POINT QAMATS
HEBREW POINT DAGESH OR MAPIQ
HEBREW POINT SHIN DOT
HEBREW LETTER MEM
HEBREW POINT PATAH
HEBREW LETTER YOD
HEBREW POINT HIRIQ
HEBREW LETTER FINAL MEM
et => sh
HEBREW LETTER SHIN
HEBREW POINT QAMATS
HEBREW POINT DAGESH OR MAPIQ
HEBREW POINT SHIN DOT
HEBREW LETTER MEM
HEBREW POINT PATAH
HEBREW LETTER YOD
HEBREW POINT HIRIQ
HEBREW LETTER MEM
11229 => hāyû
HEBREW LETTER HE
HEBREW POINT QAMATS
HEBREW LETTER YOD
HEBREW POINT METEG
HEBREW LETTER VAV
HEBREW POINT DAGESH OR MAPIQ
11230 => hāyû
HEBREW LETTER HE
HEBREW POINT QAMATS
HEBREW LETTER YOD
HEBREW LETTER VAV
HEBREW POINT DAGESH OR MAPIQ
11231 => hāʸû
HEBREW LETTER HE
HEBREW POINT QAMATS
HEBREW LETTER YOD
HEBREW LETTER VAV
HEBREW POINT DAGESH OR MAPIQ
11146 => hāʸû
HEBREW LETTER HE
HEBREW POINT QAMATS
HEBREW POINT METEG
HEBREW LETTER YOD
HEBREW LETTER VAV
HEBREW POINT DAGESH OR MAPIQ
True


In [96]:
def strip_accents(h):
    return h.translate(delchars_map)

def phono(wh, debug=False):
    if debug: 
        for u in wh:
            print(unicodedata.name(u))

    result = normalize(unorm, wh)
    result = strip_accents(result)
    if debug: 
        print('plain:      {}'.format(result))
        for u in result:
            print(unicodedata.name(u))
    if result in bolwords:
        return bolwords[result]
    if ' ' in result:
        return ' '.join(bolwords.get(part, '['+part+']') for part in result.split(' '))
    else:
        return bolwords.get(result, '['+result+']')

In [97]:
btests = (
    ('Genesis 1:15', 'הָי֤וּ'),
    ('Genesis 14:7', 'עֵ֤ין מִשְׁפָּט֙'),
    ('Genesis 14:7', 'חַֽצְצֹ֥ן')
)
for (passage, wh) in btests:
    print('{:<30} {}\n{:<30} {}'.format(passage, wh, '=>', phono(wh, True)))

HEBREW LETTER HE
HEBREW POINT QAMATS
HEBREW LETTER YOD
HEBREW ACCENT MAHAPAKH
HEBREW LETTER VAV
HEBREW POINT DAGESH OR MAPIQ
plain:      הָיוּ
HEBREW LETTER HE
HEBREW POINT QAMATS
HEBREW LETTER YOD
HEBREW LETTER VAV
HEBREW POINT DAGESH OR MAPIQ
Genesis 1:15                   הָי֤וּ
=>                             הָיוּ
HEBREW LETTER AYIN
HEBREW POINT TSERE
HEBREW ACCENT MAHAPAKH
HEBREW LETTER YOD
HEBREW LETTER FINAL NUN
SPACE
HEBREW LETTER MEM
HEBREW POINT HIRIQ
HEBREW LETTER SHIN
HEBREW POINT SHEVA
HEBREW POINT SHIN DOT
HEBREW LETTER PE
HEBREW POINT QAMATS
HEBREW POINT DAGESH OR MAPIQ
HEBREW LETTER TET
HEBREW ACCENT PASHTA
plain:      עֵיןמִשְׁפָּט
HEBREW LETTER AYIN
HEBREW POINT TSERE
HEBREW LETTER YOD
HEBREW LETTER FINAL NUN
HEBREW LETTER MEM
HEBREW POINT HIRIQ
HEBREW LETTER SHIN
HEBREW POINT SHEVA
HEBREW POINT SHIN DOT
HEBREW LETTER PE
HEBREW POINT QAMATS
HEBREW POINT DAGESH OR MAPIQ
HEBREW LETTER TET
Genesis 14:7                   עֵ֤ין מִשְׁפָּט֙
=>                             [עֵ

In [89]:
btests = range(12)
for w in btests:
    bk = F.book.v(L.u('book', w))
    ch = F.chapter.v(L.u('chapter', w))
    vs = F.verse.v(L.u('verse', w))
    wh = F.g_word_utf8.v(w)
    print('{:<30} {}\n{:<30} {}'.format('{} {}:{}'.format(bk, ch, vs), wh, '=>', phono(wh, True)))

HEBREW LETTER BET
HEBREW POINT DAGESH OR MAPIQ
HEBREW POINT SHEVA
plain:      בְּ
HEBREW LETTER BET
HEBREW POINT SHEVA
HEBREW POINT DAGESH OR MAPIQ
Genesis 1:1                    בְּ
=>                             bᵊ
HEBREW LETTER RESH
HEBREW POINT TSERE
HEBREW LETTER ALEF
HEBREW LETTER SHIN WITH SHIN DOT
HEBREW POINT HIRIQ
HEBREW ACCENT TIPEHA
HEBREW LETTER YOD
HEBREW LETTER TAV
plain:      רֵאשִׁית
HEBREW LETTER RESH
HEBREW POINT TSERE
HEBREW LETTER ALEF
HEBREW LETTER SHIN
HEBREW POINT HIRIQ
HEBREW POINT SHIN DOT
HEBREW LETTER YOD
HEBREW LETTER TAV
Genesis 1:1                    רֵאשִׁ֖ית
=>                             rēˀšît
HEBREW LETTER BET
HEBREW POINT DAGESH OR MAPIQ
HEBREW POINT QAMATS
HEBREW LETTER RESH
HEBREW POINT QAMATS
HEBREW ACCENT MUNAH
HEBREW LETTER ALEF
plain:      בָּרָא
HEBREW LETTER BET
HEBREW POINT QAMATS
HEBREW POINT DAGESH OR MAPIQ
HEBREW LETTER RESH
HEBREW POINT QAMATS
HEBREW LETTER ALEF
Genesis 1:1                    בָּרָ֣א
=>                             bārāˀ

# Print the whole text

In [90]:
msg('Generating complete texts (transcribed and phonetic) ... ')
phono_file = outfile('phono.txt')
orig_file = outfile('orig.txt')

for v in F.otype.s('verse'):
    passage_label = '{} {}:{}'.format(
        F.book.v(L.u('book', v)), 
        F.chapter.v(L.u('chapter', v)),
        F.verse.v(v),
    )
    phono_file.write('{}  '.format(passage_label))
    orig_file.write('{}  '.format(passage_label))

    words = L.d('word', v)
    verse_text = ''
    for w in words:
        the_text = F.g_word_utf8.v(w)
        the_phono = phono(the_text)
        the_trailer = F.trailer_utf8.v(w)
        the_sep = '-' if '־' in the_trailer else ': ' if '׃' in the_trailer else ' ' if ' ' in the_trailer else ''
        the_newline = '\n' if '\n' in the_trailer else ''
        verse_text += the_phono + the_sep + the_newline
    if the_sep == ' ':
        verse_text = verse_text.rstrip(' ')
    overse_text = ''.join('{}{}'.format(F.g_word_utf8.v(w), F.trailer_utf8.v(w)) for w in words)
    phono_file.write(verse_text)
    orig_file.write(overse_text)

phono_file.close()
orig_file.close()

msg('Done')

 3h 25m 58s Generating complete texts (transcribed and phonetic) ... 
 3h 26m 04s Done


# Compile "phonetic" data

# Facts

Every line contains at most one *verb*.

In [44]:
msg('Generating phonetic data file ...')

phono_file = outfile("phono.csv")

prs_map = {
    'W':  '3sm',
    'K':  '2sm',
    'J':  '1sc',
    'M':  '3pm',
    'H':  '3sf',
    'HM': '3pm',
    'KM': '2pm',
    'NW': '1pc',
    'HW': '3sm',
    'NJ': '1sc',
    'K=': '2sf',
    'HN': '3pf',
    'MW': '3pm',
    'N':  '3pf',
    'KN': '2pf',
}

fmt_str = '{}\t'+('_'.join(['{}'] * 11)) + '\n'

png = dict(
    NA='',
    unknown='',
    p1='1',
    p2='2',
    p3='3',
    sg='s',
    du='d',
    pl='p',
    m='m',
    f='f',
    a='a',
    c='c',
    e='e',
)
undefs = {'NA', 'unknown'}

for v in F.otype.s('verse'):
    passage_label = '{}\t{}\t{}'.format(
        F.book.v(L.u('book', v)).replace('_', ' '), 
        F.chapter.v(L.u('chapter', v)),
        F.verse.v(v),
    )
    lines = []
    words = L.d('word', v)
    cur_line = []
    cur_sep = ''
    skip = False
    for w in words:
        if F.language.v(w) == 'Aramaic':
            skip = True
            break
        the_trailer = F.trailer_utf8.v(w)
        if cur_sep == '':
            cur_line.append(w)
        else:
            if cur_line:
                lines.append(cur_line)
            cur_line = [w]
        cur_sep = ' ' if '־' in the_trailer else ' ' if '׃' in the_trailer else ' ' if ' ' in the_trailer else ''
    if skip:
        continue
    if cur_line:
        lines.append(cur_line)
    
    for line in lines:
        line_text = ''
        for (i,w) in enumerate(line):
            the_text = F.g_word_utf8.v(w)
            the_trailer = F.trailer_utf8.v(w)
            the_sep = '-' if '־' in the_trailer else '\t' if '׃' in the_trailer else ' ' if ' ' in the_trailer else ''
            the_newline = '\n' if '\n' in the_trailer else ''
            line_text += trans.from_hebrew(the_text) + (the_sep if i != len(line) - 1 else '')
        lex_ident = '~'.join(F.lex.v(w).replace('_', '#') for w in line)
        verb_stem = '%'.join(F.vs.v(w) for w in line if F.sp.v(w) == 'verb')
        verb_tense = '%'.join(F.vt.v(w) for w in line if F.sp.v(w) == 'verb')
        png_verb_proto = '%'.join('{}{}{}'.format(
            png[F.ps.v(w)], png[F.nu.v(w)], png[F.gn.v(w)],
        ) for w in line if F.sp.v(w) == 'verb')
        png_verb = ('V' if len(png_verb_proto) else '')+png_verb_proto
        png_noun_proto = '%'.join('{}{}'.format(
            png[F.nu.v(w)], png[F.gn.v(w)],
        ) for w in line if F.sp.v(w) == 'subs')
        png_noun = ('N' if len(png_noun_proto) else '')+png_noun_proto
        png_adjv_proto = '%'.join('{}{}'.format(
            png[F.nu.v(w)], png[F.gn.v(w)],
        ) for w in line if F.sp.v(w) == 'adjv')
        png_adjv = ('A' if len(png_adjv_proto) else '')+png_adjv_proto
        nom_st = '%'.join(png[F.st.v(w)] for w in line if F.sp.v(w) == 'subs')
        uvf_h = '~'.join(F.uvf.v(w) for w in line if F.uvf.v(w) == 'H')
        prs_noun_proto = '%'.join(prs_map.get(F.prs.v(w), '') for w in line if F.sp.v(w) == 'subs')
        prs_verb_proto = '%'.join(prs_map.get(F.prs.v(w), '') for w in line if F.sp.v(w) == 'verb')
        prs_noun = ('n' if len(prs_noun_proto) else '')+prs_noun_proto
        prs_verb = ('v' if len(prs_verb_proto) else '')+prs_verb_proto

        phono_file.write(fmt_str.format(
            passage_label, 
            phono(line_text),
            lex_ident,
            verb_stem,
            verb_tense,
            png_verb,
            png_noun,
            nom_st,
            png_adjv,
            uvf_h,
            prs_noun,
            prs_verb,
        ))
phono_file.close()
msg('Done')

 2h 30m 57s Generating phonetic data file ...
 2h 31m 38s Done


1. word per regel
2. alle features gescheiden door underscore
3. eerder aanwezige underscores vervangen door iets anders

Harald gaat splitsen in triphones
Voor elke triphone: kijken welke woorden aan die triphone deel hebben
De betekenissen van die triphones zijn de voorspellers
Je bouwt een graaf van triphones.
Een woord uitspreken is het lopen door de graaf.

Bijproduct: phonologische representatie van het Hebreeuws

Precieze codering van de pronominale suffixen


book chapter verse word features

alle feature values verschillend (dus pl for verbs different from pl for nouns)

NB: the tetragrammaton, etc.